In [1]:
from extract_features import *
from utils import *
from preprocessing import *
from sklearn.metrics import roc_curve, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

Using TensorFlow backend.
/home/skillPredict/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/skillPredict/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/skillPredict/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/skillPredict/.local/lib/python3.6/site-pa

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3909787306890965122
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17418539721835567903
physical_device_desc: "device: XLA_CPU device"
]
[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4061707132601865226
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17078176358761946701
physical_device_desc: "device: XLA_CPU device"
]
[]


In [2]:
embedding_matrix, train_seq, val_seq, y_train, y_val = preprocess(dataset='vqa', 
                                                                  n_classes=2, 
                                                                  verbose=True, 
                                                                  pretrained_embedding=False)
val_data = (val_seq, y_val)

Dataset: vqa 
Features: ['QSN', 'descriptions', 'tags', 'dominant_colors', 'handwritten_text', 'ocr_text']


W0828 03:56:54.570125 140136212100928 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


Total 872 word vectors
Number of training samples each class: 
Text recognition - 1: 358 0: 2865
Color recognition - 1:1145 0: 2078
Counting - 1: 661 0: 2562
Number of validation samples each class: 
Text recognition - 1: 63 0: 450
Color recognition - 1:179 0: 334
Counting - 1:109 0: 404
PCA with 40 eigenvectors


In [6]:
embedding_matrix[1]

array([ 6.54482484e-01,  1.06252789e-01,  4.42153037e-01,  1.84124351e-01,
        1.38634264e-01, -1.07114285e-01, -1.07097149e-01, -2.57948160e-01,
       -3.00118122e-02, -4.07301784e-02,  1.94145575e-01, -5.83471581e-02,
        3.47945333e-01, -2.61357248e-01,  1.30560547e-01,  5.05380332e-02,
       -8.39845911e-02,  1.67767882e-01,  1.81281105e-01, -1.21363141e-01,
       -1.71907008e-01, -1.72970429e-01,  2.13397458e-01,  1.94949470e-02,
       -1.14836013e-02, -3.61826301e-01,  4.75321561e-02, -2.49433726e-01,
       -5.57328284e-01, -3.77679057e-02, -1.73015952e-01,  2.48564765e-01,
        2.23067757e-02, -1.23598084e-01,  1.68508708e-01, -1.40102193e-01,
       -2.86425911e-02, -2.17689931e-01, -1.64651051e-01, -1.10583365e-01,
        4.23390538e-01,  2.60230690e-01, -2.74226516e-01, -1.40046537e-01,
        3.04138839e-01,  3.94218296e-01,  7.21935229e-03, -4.27828804e-02,
        7.06532747e-02,  3.04483771e-01, -2.36380726e-01, -7.41376542e-03,
        9.30968001e-02,  

In [3]:
L = 1e-3
R = 1e-7
B = 32
E = 500

model, history = skill_predictor(train_seq, embedding_matrix,
                 train_labels=y_train,
                 val_data=(val_seq,y_val),
                 learning_rate=L,
                 lstm_dim=100,
                 batch_size=B,
                 num_epochs=E,
                 optimizer_param=SGD(lr=L, nesterov=True),
                 n_classes=2,
                 regularization=R, verbose=1)

W0828 03:56:54.646769 140136212100928 deprecation_wrapper.py:119] From /home/skillPredict/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 03:56:55.321100 140136212100928 deprecation_wrapper.py:119] From /home/skillPredict/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0828 03:56:55.330434 140136212100928 deprecation.py:323] From /home/skillPredict/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Log file: ./LSTM/0.001_1e-07_32_500.log
Train on 3223 samples, validate on 513 samples
Epoch 1/500


InvalidArgumentError: indices[13,26] = -2 is not in [0, 3000)
	 [[{{node embedding_1/embedding_lookup}}]]

In [ ]:
preds = model.predict(val_seq, verbose=0)
y_pred_text = [1 if i >= 0.5 else 0 for i in preds[:,0]]
y_pred_color = [1 if i >= 0.5 else 0 for i in preds[:,1]]
print("VizWiz, all features - accuracy\ntext: {}\ncolor:{}".format(accuracy_score(y_val[:,0], y_pred_text),
                                                        accuracy_score(y_val[:,1], y_pred_color)))